In [1]:
# Library
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from collections import defaultdict
from datetime import datetime
from itertools import permutations
import pickle
import os
import gc

from polygon import RESTClient
from polygon.rest.models import (
    TickerNews,
    Ticker,
    MarketHoliday
)
from polygon.exceptions import BadResponse

API_KEY = "M5o8aqT125ELXrrxJKBC5E2G6pqUSDuC"
PATH = "e:/Financial_Data/Polygon/"
client = RESTClient(API_KEY)

In [3]:
df = pd.read_feather(PATH+"inlisted_5m/0-49.feather")

In [175]:
EXCHANGES = ['XNYS', 'XNAS', 'XASE', 'ARCX', 'BATS']
ITEM_TYPES = ['CS', 'ETF', 'ETN']

In [2]:
tickers = dict()
l = []
for ex, ty in [(x, y) for x in EXCHANGES for y in ITEM_TYPES]:
    for t in client.list_tickers(type=ty, exchange=ex, active=True, limit=1000, market='stocks'):
        l.append(t.__dict__)

NameError: name 'EXCHANGES' is not defined

In [2]:
ticker_delisted_df = pd.read_csv(PATH+"ticker_delisted_info.csv")
ticker_inlisted_df = pd.read_csv(PATH+"ticker_inlisted_info.csv")
# with open(PATH+'ind.pickle', 'rb') as fr:
#   time_ind = pickle.load(fr)

In [4]:
start = 0; stop = 200000; step = 400; cur = start; prev = start
while True:
    i = 0
    aggs = dict()
    while i < step:
        try:
            t = ticker_delisted_df['ticker'][cur]    
        except: break
        cur += 1 
        if aggs.get(t, 'none') != 'none': 
            continue
        
        aggs[t] = []
        try:
            for a in client.list_aggs(
                t,
                1,
                "day",
                "2015-01-01",
                "2024-12-15",
                limit=50000,
                adjusted=True,
            ):
                aggs[t].append(a.__dict__)
        except BadResponse: 
            del aggs[t]
            continue
        # print(datetime.fromtimestamp(aggs[t][0]['timestamp']/1000), datetime.fromtimestamp(aggs[t][-1]['timestamp']/1000))
        
        if len(aggs[t]) == 0:
            del aggs[t]
        else:
            i += 1
            # with open(PATH+"inlisted_1d/timerange.txt", "a") as file:
            #     file.write(f"{t} {aggs[t][0]['timestamp']//1000} {aggs[t][-1]['timestamp']//1000}\n")
            
    print(aggs.keys())
        
    dfl = []
    for k, v in aggs.items():
        df = pd.DataFrame(v)
        df['timestamp'] = df['timestamp'] // 1000
        df = df.set_index('timestamp')
        df = df.drop(columns=['otc'])
        df.columns = pd.MultiIndex.from_product([[k], df.columns])
        dfl.append(df)
    
    concated = pd.concat(dfl, axis=1, join='outer')
    # concated = concated.reindex(time_ind)
    concated.to_feather(PATH+f'delisted_1d/{prev}-{cur-1}.feather')
    
    prev = cur
    
    if cur >= stop: break  

dict_keys(['AA', 'AAC', 'AAIC', 'AAN', 'AANw', 'AAQC', 'AAV', 'AAw', 'ABB', 'ABY', 'AC', 'AC.WD', 'ACA', 'ACAw', 'ACB', 'ACC', 'ACDI', 'ACE', 'ACG', 'ACH', 'ACI', 'ACIC', 'ACII', 'ACMP', 'ACN', 'ACND', 'ACP', 'ACR', 'ACRO', 'ACT', 'ACTV', 'ACV', 'ACW', 'ADF', 'ADNTw', 'ADPT', 'ADSW', 'ADSw', 'ADT', 'AEC', 'AED', 'AEG', 'AEL', 'AESI', 'AET', 'AEVA', 'AF', 'AFC', 'AFI', 'AFIw', 'AFTR', 'AGAC', 'AGC', 'AGCB', 'AGD', 'AGE', 'AGI', 'AGL', 'AGN', 'AGTI', 'AGU', 'AHC', 'AHG', 'AHL', 'AHP', 'AHR', 'AHT', 'AI', 'AIB', 'AIC', 'AIG', 'AIQ', 'AIRC', 'AIV', 'AJAX', 'AJRD', 'AJX', 'AKA', 'AKP', 'AKS', 'AL', 'ALC', 'ALD', 'ALDW', 'ALE', 'ALEX', 'ALJ', 'ALLG', 'ALO', 'ALR', 'ALU', 'ALUS', 'ALVw', 'AM', 'AMBO', 'AMBR', 'AMC', 'AMFW', 'AMGP', 'AMH', 'AMID', 'AMK', 'AMN', 'AMPI', 'AMR', 'AMRE', 'AMT', 'AMTG', 'AMTMw', 'AMU', 'AMWL', 'AMX', 'ANAC', 'ANDV', 'ANDX', 'ANET', 'ANFI', 'ANH', 'ANL', 'ANN', 'ANR', 'ANW', 'AOD', 'AOI', 'AOL', 'AON', 'AONE', 'APA', 'APB', 'APC', 'APCA', 'APF', 'APFH', 'APGB', 'APL

ValueError: No objects to concatenate

In [20]:
temp = list()
for a in client.list_aggs(
    'AAPL',
    1,
    "day",
    "2015-01-01",
    "2024-12-15",
    limit=50000,
    adjusted=True,
):
    temp.append(a.__dict__)
temp = pd.DataFrame(temp)

In [23]:
temp['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x//1000))

0      2015-01-02 05:00:00
1      2015-01-05 05:00:00
2      2015-01-06 05:00:00
3      2015-01-07 05:00:00
4      2015-01-08 05:00:00
               ...        
2500   2024-12-09 05:00:00
2501   2024-12-10 05:00:00
2502   2024-12-11 05:00:00
2503   2024-12-12 05:00:00
2504   2024-12-13 05:00:00
Name: timestamp, Length: 2505, dtype: datetime64[ns]

In [14]:
l = [concated[k] for k in concated.columns.get_level_values(0).unique()]

In [ ]:
# Test
i = 0
aggs = dict()
t = 'A'  
aggs[t] = []
try:
    for a in client.list_aggs(
        t,
        1,
        "minute",
        "2015-01-01",
        "2024-12-13",
        limit=50000,
        adjusted=True,
    ):
        aggs[t].append(a.__dict__)
except BadResponse: 
    del aggs[t]
# print(datetime.fromtimestamp(aggs[t][0]['timestamp']/1000), datetime.fromtimestamp(aggs[t][-1]['timestamp']/1000))

if len(aggs[t]) == 0:
    del aggs[t]
else:
    i += 1
        
print(aggs.keys())

dict_keys(['A'])


In [9]:
p = PATH + 'delisted_5m'
all_files = os.listdir(p)  # 현재 디렉토리의 모든 파일과 폴더 이름

# 파일 이름만 필터링
file_names = [f for f in all_files if os.path.isfile(os.path.join(p, f))]

In [10]:
dataf = dict()
for fn in file_names:
  temp = pd.read_feather(p+f'/{fn}')
  indices_list = temp.columns.levels[0].tolist()
  dataf[fn] = indices_list
  _ = gc.collect()

In [11]:
df = pd.DataFrame(dataf)
df.T.to_csv(p+'/_delisted_ticker_df.csv')

In [15]:
trades = []
for t in client.list_trades("SOUN", timestamp_gt='2022-01-05', timestamp_lte='2022-05-02', limit=50000):
    trades.append(t)

# prints each trade that took place
# print(trades)

In [16]:
len(trades)

109443

In [9]:
conditions = []
for c in client.list_conditions(limit=1000, asset_class='stocks'):
    conditions.append(c.__dict__)

In [10]:
pd.DataFrame.from_records(conditions).to_csv(PATH+'condition.csv')

In [66]:
dfl = []
for k, v in aggs.items():
  df = pd.DataFrame(v)
  df['timestamp'] = df['timestamp'] // 1000
  df = df.set_index('timestamp')
  df = df.drop(columns=['otc'])
  df.columns = pd.MultiIndex.from_product([[k], df.columns])
  dfl.append(df)

In [79]:
concated = pd.concat(dfl, axis=1)
concated = concated.reindex(time_ind)
concated.iloc[28:]

AAC                                                         ACC  \
             open    high    low   close  volume     vwap transactions   open   
timestamp                                                                       
1420208400    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   
1420209000  30.87  30.870  30.87  30.870   469.0  30.8517          7.0  41.59   
1420209600  30.95  31.280  30.95  31.270  2000.0  31.1557         19.0  41.45   
1420210200  31.49  31.580  31.42  31.500  1100.0  31.4949         11.0  41.53   
1420210800  31.44  31.571  31.41  31.571   904.0  31.4510          8.0  41.55   
...           ...     ...    ...     ...     ...      ...          ...    ...   
1734091200    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   
1734091800    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   
1734092400    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   
1734093000    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   
1734093600    NaN     NaN    NaN     NaN     NaN      NaN          NaN    NaN   

                                                                 
             high     low  close   volume     vwap transactions  
timestamp                                                        
1420208400    NaN     NaN    NaN      NaN      NaN          NaN  
1420209000  41.59  41.384  41.45  12564.0  41.5456         96.0  
1420209600  41.65  41.430  41.54  16640.0  41.5292        147.0  
1420210200  41.55  41.440  41.49  11415.0  41.5083        119.0  
1420210800  41.77  41.525  41.60   4603.0  41.6279         57.0  
...           ...     ...    ...      ...      ...          ...  
1734091200    NaN     NaN    NaN      NaN      NaN          NaN  
1734091800    NaN     NaN    NaN      NaN      NaN          NaN  
1734092400    NaN     NaN    NaN      NaN      NaN          NaN  
1734093000    NaN     NaN    NaN      NaN      NaN          NaN  
1734093600    NaN     NaN    NaN      NaN      NaN          NaN  

[229115 rows x 14 columns]

In [41]:
df.set_index('timestamp')

,open,high,low,close,volume,vwap,transactions,otc
timestamp,,,,,,,,
1418740200000,29.770,30.0000,29.3700,30.0000,1201.0,29.7733,5,None
1418740800000,29.916,30.3897,29.8301,30.3897,4873.0,29.9250,29,None
1418741400000,30.110,30.5199,30.1100,30.3000,3365.0,30.3273,30,None
1418742000000,30.520,30.5700,30.3100,30.3800,971.0,30.4017,9,None
1418742600000,30.200,30.2000,29.5700,29.7150,2458.0,29.9705,27,None
...,...,...,...,...,...,...,...,...
1699297800000,10.790,10.7900,10.7900,10.7900,100.0,10.7900,1,None
1699298400000,10.785,10.7850,10.7850,10.7850,289.0,10.7850,1,None
1699300800000,10.785,10.7850,10.7850,10.7850,600.0,10.7850,6,None


In [30]:
aggs[0].__dict__

{'open': 26.975,
 'high': 26.975,
 'low': 26.975,
 'close': 26.975,
 'volume': 17660,
 'vwap': 26.975,
 'timestamp': 1418720400000,
 'transactions': 9,
 'otc': None}

In [29]:
client.get_previous_close_agg('AAC')

[PreviousCloseAgg(ticker='AAC', close=10.79, high=10.79, low=10.77, open=10.77, timestamp=1699304400000, volume=599704, vwap=10.7802)]

In [44]:
types = client.get_ticker_types()
types

[TickerTypes(asset_class='stocks', code='CS', description='Common Stock', locale='us'),
 TickerTypes(asset_class='stocks', code='PFD', description='Preferred Stock', locale='us'),
 TickerTypes(asset_class='stocks', code='WARRANT', description='Warrant', locale='us'),
 TickerTypes(asset_class='stocks', code='RIGHT', description='Rights', locale='us'),
 TickerTypes(asset_class='stocks', code='BOND', description='Corporate Bond', locale='us'),
 TickerTypes(asset_class='stocks', code='ETF', description='Exchange Traded Fund', locale='us'),
 TickerTypes(asset_class='stocks', code='ETN', description='Exchange Traded Note', locale='us'),
 TickerTypes(asset_class='stocks', code='ETV', description='Exchange Traded Vehicle', locale='us'),
 TickerTypes(asset_class='stocks', code='SP', description='Structured Product', locale='us'),
 TickerTypes(asset_class='stocks', code='ADRC', description='American Depository Receipt Common', locale='us'),
 TickerTypes(asset_class='stocks', code='ADRP', descrip